# Assignment 2. Shumovskaia Valentina.

In [1]:
from cvxpy import *
import numpy as np

## Excercise 1.4

#### Consider the problem
$$
\textrm{minimize }4x_1 +5\left|x_2-1\right|\\
\textrm{subject to }\left|2x_1\right|+\left|x_2-3\right|\leq 5
$$
#### and reformulate it as a linear programming problem.

You may consider any $x \in \mathbb{R}$ as $x=x^+ -x^-$, where $x^+, x^- \geq 0$. Then $\left|x\right| = x^++x^-.$

1. Let's look at $x_1$ as $x_1 = s_1 - s_2$, then $\left|2x_1\right| = 2s_1 + 2s_2,$ where $s_1,s_2 \geq 0.$

2. Let's consider $\left|x_2-1\right|$. $x_2-1=s_3-s_4$, then $x_2 = 1+s_3-s_4$ and $\left|x_2-1\right| = s_3+s_4$, where $s_3,s_4\geq 0$.

3. Let's condider $\left|x_2-3\right|$. $x_2-3=s_5-s_6$, then $x_2 = 3+s_5-s_6$ and $\left|x_2-3\right| = s_5+s_6$, where $s_5,s_6\geq 0$.

4. Also we need to add a new constraint because we can compute $x_2$ in two ways. So, $1+s_3-s_4 = 3+s_5-s_6 <=> s_3-s_4-s_5+s_6=2.$

Thus, our new problem is (in inequality form):

$$
\textrm{minimize}\\ 4x_1 + 5s_3 + 5s_4\\
\textrm{subject to}\\ 2s_1 + 2s_2 + s_5 + s_6 \leq 5,\\
x_1 - s_1 + s_2 \leq 0,\\
-x_1 + s_1 - s_2 \leq 0,\\
x_2 - s_3 + s_4 \leq 1,\\
- x_2 + s_3 - s_4 \leq -1,\\
s_3-s_4-s_5+s_6 \leq 2,\\
-s_3+s_4+s_5-s_6 \leq -2,\\
-s_1,-s_2,-s_3,-s_4,-s_5,-s_6\leq 0.
$$

An optimal value for initial task would be optimal value for a new task.

#### CVXPY soluton

In [2]:
x_1 = Variable()
x_2 = Variable()
s_1 = Variable()
s_2 = Variable()
s_3 = Variable()
s_4 = Variable()
s_5 = Variable()
s_6 = Variable()

constraints = [-s_1 <= 0,
               -s_2 <= 0,
               -s_3 <= 0,
               -s_4 <= 0,
               -s_5 <= 0,
               -s_6 <= 0,
               x_1 - s_1 + s_2 <= 0,
               -x_1 + s_1 - s_2 <= 0,
               x_2 - s_3 + s_4 <= 1,
               -x_2 + s_3 - s_4 <= -1,
               2*s_1 + 2*s_2 + s_5 + s_6 <= 5,
               s_3 - s_4 - s_5 + s_6 <= 2,
               -s_3 + s_4 + s_5 - s_6 <= -2
              ]

obj = Minimize(4*s_1 - 4*s_2 + 5*s_3 + 5*s_4)

prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value =", prob.value)
print("x_1* =", x_1.value)
print("x_2* =", x_2.value)

status: optimal
optimal value = -5.999999998898642
x_1* = -1.50000000055
x_2* = 1.00000000085


## Excercise 1.8

#### Consider a road divided into $n$ segments and illuminated by $m$ lamps. Let $p_j$ be the power of the $j$th lamp. The illumination $I_i$ of the $i$th segment is assumed to be $\sum_{j=1}^ma_{ij}p_j$, where $a_{ij}$ are known coefficients. Let $I_i^*$ be the desired illumination of road $i$.

#### We are interested in choosing the lamp powers $p_j$ so that the illuminations $I_i$ are close to the desired illuminations $I^∗_i$. Provide a reasonable linear programming formulation of this problem. Note that the wording of the problem is loose and there is more than one possible formulation.

We want $I_i$ to be close to $I_i^*$, so let's minimize $\sum_{i=1}^m\left|I_i-I_i^*\right|$ knowing that $I_i = \sum_{j=1}^na_{ij}p_j = I_j$ and $p_j \geq 0$ for $j=1,\dots,n$. As in previous task we need to divide absolute values into two parts $\left|I_i-I_i^*\right| = s_{1_i}^+ + s_{2_i}^-$. So, we're adding two new variables $s_1$ and $s_2$ which are positive vecors size $n$, also consider $I$ and $I^*$ as vectors: $I = (I_1,\dots,I_n)^T, I^* = (I^*_1,\dots,I^*_n)^T$. Let's also denote $p = (p_1,\dots,p_n)^T$, $A = (a_{ij})_{i=1,\dots,m,j=1,\dots,n}).$

Linear programming problem:
$$
\textrm{minimize}\\
\sum_{i=1}^m s_{1_i} - s_{2_i}\\
\textrm{subject to}\\
Ap \leq I,\\
-Ap \leq -I,\\
I - I^* - s_1 + s_2 \leq 0,\\
- I + I^* + s_1 - s_2 \leq 0,\\
-s_1 \leq 0,\\
-s_2 \leq 0,\\
-p \leq 0.
$$

#### CVXPY solution

In [5]:
n = 20
m = 10
I_desire = np.ones(m)*5
#A = np.ones((m,n))
A = np.random.rand(m,n)

p = Variable(n)
I = Variable(m)
s_1 = Variable(m)
s_2 = Variable(m)

constraints = [A@p <= I,
               -A@p <= -I,
               I - I_desire - s_1 + s_2 <= 0,
               - I + I_desire + s_1 - s_2 <= 0,
               -s_1 <= 0,
               -s_2 <= 0,
               -p <= 0
              ]

obj = Minimize(sum((I-I_desire)**2))

prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value =", prob.value)
print("p* =", p.value)
print("I = ", I.value)
print("I^* = ", I_desire)

status: optimal
optimal value = -7.70296846163975e-11
p* = [[ 0.15168488]
 [ 0.37475207]
 [ 0.12086434]
 [ 0.50028867]
 [ 0.20788085]
 [ 0.762462  ]
 [ 0.79860596]
 [ 0.67139863]
 [ 0.66966923]
 [ 0.11788   ]
 [ 1.4048173 ]
 [ 0.29831977]
 [ 1.47349953]
 [ 0.19852284]
 [ 0.22804621]
 [ 0.11672018]
 [ 0.25744904]
 [ 0.17549554]
 [ 0.25842703]
 [ 1.29224287]]
I =  [[ 5.0000001 ]
 [ 4.99999938]
 [ 5.00000007]
 [ 4.99999977]
 [ 4.99999951]
 [ 5.00000005]
 [ 4.99999994]
 [ 5.00000072]
 [ 5.00000002]
 [ 4.99999989]]
I^* =  [ 5.  5.  5.  5.  5.  5.  5.  5.  5.  5.]


## Excercise 1.15

#### A company produces two kinds of products. A product of the first type requires $1/4$ hours of assembly labor, $1/8$ hours of testing, and $\$1.2$ worth of raw materials. A product of the second type requires $1/3$ hours of assembly, $1/3$ hours of testing, and $\$0.9$ worth of raw materials. Given the current personnel of the company, there can be at most $90$ hours of assimbly labor and $80$ hours of testing, each day. Products of the first and second type have a market value of $\$9$ and $\$8$, perspectively.

#### 1. Formulate a linear programming problem that can be used to maximize the daily profit of the company.

#### 2. Consider the following two modifications to the original problem:

#### (a) Suppose that up to $50$ hours of overtime assembly labor can be scheduled, at a cost of $\$7$ per hour.

#### (b) Suppose that the raw material supplier provides a $10\%$ discount if the daily bill is above $\$300$. 

#### Which of the above two elements can be easily incorporated into the linear programming formulation and how? If one or both are not easy to incorporate, indicate how you might nevertheless solve the problem.

Let's denote $n_1, n_2$ -- amount of first and second type products. We want to maximize the following sum: $n_1\cdot (9-1.2) + n_2\cdot (8-0.9)$ remembering that $n_1, n_2 \geq 0.$ Also to fulfil time requirements we should add constraints: $n_1\cdot \frac{1}{4} + n_2\cdot \frac{1}{3} \leq 90, n_1\cdot \frac{1}{8} + n_2\cdot \frac{1}{3} \leq 80$.

Linear programming problem:
$$
\textrm{minimize}\\
n_1\cdot (9-1.2) + n_2 \cdot (8-0.9)\\
\textrm{subject to}\\
n_1\cdot \frac{1}{4} + n_2\cdot \frac{1}{3} \leq 90,\\
n_1\cdot \frac{1}{8} + n_2\cdot \frac{1}{3} \leq 80,\\
-n_1 \leq 0,\\
-n_2 \leq 0.\\
$$

#### CVXPY solution

In [4]:
n_1 = Variable()
n_2 = Variable()

constraints = [-n_1 <= 0,
               -n_2 <= 0,
               n_1/4 + n_2/3 <= 90,
               n_1/8 + n_2/3 <= 80,
              ]

obj = Maximize(n_1*(9-1.2) + n_2*(8-0.9))

prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value =", prob.value)
print("n_1* =", n_1.value)
print("n_2* =", n_2.value)

sol_1 = prob.value

status: optimal
optimal value = 2807.999988249077
n_1* = 359.999995524
n_2* = 3.26239654109e-06


(a) the problem easily modifies by adding a new variable $t$ such that $0 \leq t \leq 5$. We change the objetive funtion into $n_1\cdot (9-1.2) + n_2 \cdot (8-0.9) - t\cdot 7$ and change one of the constraints: $n_1/4 + n_2/3 <= 90 + t.$

Thus, linear programming problem looks like:
$$
\textrm{minimize}\\
n_1\cdot (9-1.2) + n_2 \cdot (8-0.9) + t \cdot 7\\
\textrm{subject to}\\
n_1\cdot \frac{1}{4} + n_2\cdot \frac{1}{3} \leq 90 + t,\\
n_1\cdot \frac{1}{8} + n_2\cdot \frac{1}{3} \leq 80,\\
-n_1 \leq 0,\\
-n_2 \leq 0,\\
-t \leq 0,\\
t \leq 50.
$$

#### CVXPY solution

In [5]:
n_1 = Variable()
n_2 = Variable()
t = Variable()

constraints = [-n_1 <= 0,
               -n_2 <= 0,
               -t <= 0,
               t <= 50,
               n_1/4 + n_2/3 <= 90 + t,
               n_1/8 + n_2/3 <= 80,
              ]

obj = Maximize(n_1*(9-1.2) + n_2*(8-0.9) - t*7)

prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value =", prob.value)
print("n_1* =", n_1.value)
print("n_2* =", n_2.value)

status: optimal
optimal value = 4017.999997922294
n_1* = 559.999999348
n_2* = 4.04503845293e-07


(b) in this case it's not easy to modify a problem, but we can solve 2 problems, an initial one and the following one, compare their results and choose the best one.

We need to consider that we have $10\%$discount and that a daily bill is above $300\$$. Thus, second linear programming problem is\\
$$
\textrm{minimize}\\
n_1\cdot (9-1.2*0.9) + n_2 \cdot (8-0.9*0.9)\\
\textrm{subject to}\\
n_1\cdot \frac{1}{4} + n_2\cdot \frac{1}{3} \leq 90,\\
n_1\cdot \frac{1}{8} + n_2\cdot \frac{1}{3} \leq 80,\\
-n_1\cdot1.2 - n_2\cdot0.9 \leq -300,\\
-n_1 \leq 0,\\
-n_2 \leq 0.\\
$$

#### CVXPY solution

In [6]:
n_1 = Variable()
n_2 = Variable()

constraints = [-n_1 <= 0,
               -n_2 <= 0,
               -n_1*1.2 - n_2*0.9 <= - 300,
               n_1/4 + n_2/3 <= 90,
               n_1/8 + n_2/3 <= 80,
              ]

obj = Maximize(n_1*(9-0.9*1.2) + n_2*(8-0.9*0.9))

prob = Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value =", prob.value)
print("n_1* =", n_1.value)
print("n_2* =", n_2.value)

sol_2b = prob.value

status: optimal
optimal value = 2851.199996274337
n_1* = 359.999998359
n_2* = 1.28924150276e-06


In [7]:
print("An optimal value is", max(sol_1, sol_2b))

An optimal value is 2851.199996274337


## Excercise 3.12

#### Consider the problem
$$
\textrm{minimize }\\-x_1 -2x_2\\
\textrm{subject to }\\
-x_1 + 3x_2 \leq 3,\\
x_1 + x_2 \leq 5,\\
x_1,x_2 \geq 0.
$$

#### 1. Convert the problem into standard form and construct a basic feasible solution at which $(x_1, x_2) = (0, 0)$.

#### 2. Carry out the full tableau implementation of the simplex method, starting with the basic feasible solution of part 1.

#### 3. Draw a graphical representation of the problem in terms of the original variables $x_1$, $x_2$, and indicate the path taken by the simplex algorithm.

1. We should add slack variables $x_3, x_4 \geq 0$. Then problem transposes to the following:

$$
\textrm{minimize }\\-x_1 -2x_2\\
\textrm{subject to }\\
-x_1 + 3x_2  + x_3 = 3,\\
x_1 + x_2 + x_4 = 5,\\
x_1,x_2,x_3,x_4 \geq 0.
$$

![](3_1.JPG =100x20)


<img src="3_sdsd1.JPG">


<img src="3_1.JPG" alt="Drawing" style="width: 700px;"/>
<img src="3_2.JPG" alt="Drawing" style="width: 700px;"/>